In [2]:
from pyscf import gto, scf, cc
import numpy as np
from jax import numpy as jnp
from jax import vmap

a = 1.5 # 2aB
nH = 6
atoms = ""
for i in range(nH):
    atoms += f"H {i*a:.5f} 0.00000 0.00000 \n"

mol = gto.M(atom=atoms, basis="sto3g", verbose=4)
mol.build()

mf = scf.RHF(mol)#.density_fit()
e = mf.kernel()

mycc = cc.CCSD(mf)
e = mycc.kernel()

System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#5074-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Sat Oct  4 22:03:56 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 6
[INPUT] num. electrons = 6
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.00

In [4]:
options = {'n_eql': 4,
           'n_prop_steps': 10,
            'n_ene_blocks': 1,
            'n_sr_blocks': 10,
            'n_blocks': 10,
            'n_walkers': 1,
            'seed': 2,
            'walker_type': 'rhf',
            'trial': 'cisd',
            'dt':0.005,
            'free_projection':False,
            'ad_mode':None,
            'use_gpu': False,
            }

In [6]:
from ad_afqmc import pyscf_interface, mpi_jax, driver
pyscf_interface.prep_afqmc(mycc,chol_cut=1e-7)
ham_data, ham, prop, trial, wave_data, sampler, observable, options, _ \
    = (mpi_jax._prep_afqmc(options))

#
# Preparing AFQMC calculation
# If you import pyscf cc modules and use MPI for AFQMC in the same script, finalize MPI before calling the AFQMC driver.
# Calculating Cholesky integrals
# Finished calculating Cholesky integrals
#
# Size of the correlation space:
# Number of electrons: (3, 3)
# Number of basis functions: 6
# Number of Cholesky vectors: 15
#
# Number of MPI ranks: 1
#
# norb: 6
# nelec: (3, 3)
#
# n_eql: 4
# n_prop_steps: 10
# n_ene_blocks: 1
# n_sr_blocks: 10
# n_blocks: 10
# n_walkers: 1
# seed: 2
# walker_type: rhf
# trial: cisd
# dt: 0.005
# free_projection: False
# use_gpu: False
# n_exp_terms: 6
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
# n_batch: 1
# LNO: False
# orbE: 0
# maxError: 0.001
#


In [88]:
from mpi4py import MPI
from jax import random

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
seed = options["seed"]
neql = options["n_eql"]

# trial_rdm1 = trial.get_rdm1(wave_data)
# if "rdm1" not in wave_data:
#     wave_data["rdm1"] = trial_rdm1

ham_data = ham.build_measurement_intermediates(ham_data, trial, wave_data)
ham_data = ham.build_propagation_intermediates(ham_data, prop, trial, wave_data)

init_walkers = None
prop_data = prop.init_prop_data(trial, wave_data, ham_data, init_walkers)
prop_data['n_killed_walkers'] = 0
prop_data["key"] = random.PRNGKey(seed + rank)
init_walker = prop_data['walkers'][0]
print(prop_data['e_estimate'])
print(mycc.e_tot)

-2.999851469512132
-2.9998514912772754


In [13]:
prop_data['walkers']

Array([[[-1.35869622e-33+0.j, -8.50166058e-35+0.j,  1.00000000e+00+0.j],
        [ 1.08180414e-16+0.j,  1.00000000e+00+0.j,  0.00000000e+00+0.j],
        [ 1.00000000e+00+0.j, -1.08180414e-16+0.j,  4.31408308e-32+0.j],
        [-4.74822782e-17+0.j, -9.02056208e-17+0.j,  1.39059730e-17+0.j],
        [-1.59134048e-16+0.j, -1.66533454e-16+0.j,  1.43713517e-16+0.j],
        [ 4.91004945e-17+0.j,  0.00000000e+00+0.j, -3.71730138e-16+0.j]]],      dtype=complex128)

In [14]:
def _walker_rhf_olp(walker):
    # nocc, ci1, ci2 = walker.shape[1], wave_data["ci1"], wave_data["ci2"]
    # GF = (walker.dot(jnp.linalg.inv(walker[: walker.shape[1], :]))).T
    o0 = jnp.linalg.det(walker[: walker.shape[1], :]) ** 2
    # o1 = jnp.einsum("ia,ia", ci1, GF[:, nocc:])
    # o2 = 2 * jnp.einsum(
    #     "iajb, ia, jb", ci2, GF[:, nocc:], GF[:, nocc:]
    # ) - jnp.einsum("iajb, ib, ja", ci2, GF[:, nocc:], GF[:, nocc:])
    return o0

In [ ]:
ci1, ci2 = wave_data["ci1"], wave_data["ci2"]
print(ci1.shape,ci2.shape)
# c1_ia, ci2_iajb

(3, 3) (3, 3, 3, 3)


In [19]:
from jax import lax
def _calc_cisd_walker_pert_energy(trial, walker, ham_data, wave_data):
    ci1, ci2 = wave_data["ci1"], wave_data["ci2"]
    nocc = trial.nelec[0]
    green = (walker.dot(jnp.linalg.inv(walker[:nocc, :]))).T
    green_occ = green[:, nocc:].copy()
    greenp = jnp.vstack((green_occ, -jnp.eye(trial.norb - nocc)))

    chol = ham_data["chol"].reshape(-1, trial.norb, trial.norb)
    rot_chol = chol[:, : trial.nelec[0], :]
    h1 = (ham_data["h1"][0] + ham_data["h1"][1]) / 2.0
    hg = jnp.einsum("pj,pj->", h1[:nocc, :], green)

    # 0 body energy
    e0 = ham_data["h0"]

    # 1 body energy
    # ref
    e1_0 = 2 * hg

    # single excitations
    ci1g = jnp.einsum("ia,ia->", ci1, green_occ, optimize="optimal")
    e1_1_1 = 4 * ci1g * hg
    gpci1 = greenp @ ci1.T
    ci1_green = gpci1 @ green
    e1_1_2 = -2 * jnp.einsum("ij,ij->", h1, ci1_green, optimize="optimal")
    e1_1 = e1_1_1 + e1_1_2

    # double excitations
    # ci2g_c = jnp.einsum("ptqu,pt->qu", ci2, green_occ)
    ci2g_c = jnp.einsum("iajb,ia->jb", ci2, green_occ)
    ci2g_e = jnp.einsum("iajb,ib->ja", ci2, green_occ)
    ci2_green_c = (greenp @ ci2g_c.T) @ green
    ci2_green_e = (greenp @ ci2g_e.T) @ green
    ci2_green = 2 * ci2_green_c - ci2_green_e
    ci2g = 2 * ci2g_c - ci2g_e
    gci2g = jnp.einsum("ia,ia->", ci2g, green_occ, optimize="optimal")
    e1_2_1 = 2 * hg * gci2g
    e1_2_2 = -2 * jnp.einsum("ij,ij->", h1, ci2_green, optimize="optimal")
    e1_2 = e1_2_1 + e1_2_2
    e1 = e1_0 + e1_1 + e1_2

    # two body energy
    # ref
    lg = jnp.einsum("gpj,pj->g", rot_chol, green, optimize="optimal")
    # lg1 = jnp.einsum("gpj,pk->gjk", rot_chol, green, optimize="optimal")
    lg1 = jnp.einsum("gpj,qj->gpq", rot_chol, green, optimize="optimal")
    e2_0_1 = 2 * lg @ lg
    e2_0_2 = -jnp.sum(vmap(lambda x: x * x.T)(lg1))
    e2_0 = e2_0_1 + e2_0_2

    # single excitations
    e2_1_1 = 2 * e2_0 * ci1g
    lci1g = jnp.einsum("gij,ij->g", chol, ci1_green, optimize="optimal")
    e2_1_2 = -2 * (lci1g @ lg)
    # lci1g1 = jnp.einsum("gij,jk->gik", chol, ci1_green, optimize="optimal")
    # glgpci1 = jnp.einsum(("gpi,iq->gpq"), gl, gpci1, optimize="optimal")
    ci1g1 = ci1 @ green_occ.T
    # e2_1_3 = jnp.einsum("gpq,gpq->", glgpci1, lg1, optimize="optimal")
    e2_1_3_1 = jnp.einsum("gpq,gqr,rp->", lg1, lg1, ci1g1, optimize="optimal")
    lci1g = jnp.einsum("gip,qi->gpq", ham_data["lci1"], green, optimize="optimal")
    e2_1_3_2 = -jnp.einsum("gpq,gqp->", lci1g, lg1, optimize="optimal")
    e2_1_3 = e2_1_3_1 + e2_1_3_2
    e2_1 = e2_1_1 + 2 * (e2_1_2 + e2_1_3)

    # double excitations
    e2_2_1 = e2_0 * gci2g
    lci2g = jnp.einsum("gij,ij->g", chol, ci2_green, optimize="optimal")
    e2_2_2_1 = -lci2g @ lg

    # lci2g1 = jnp.einsum("gij,jk->gik", chol, ci2_green, optimize="optimal")
    # lci2_green = jnp.einsum("gpi,ji->gpj", rot_chol, ci2_green, optimize="optimal")
    # e2_2_2_2 = 0.5 * jnp.einsum("gpi,gpi->", gl, lci2_green, optimize="optimal")
    def scanned_fun(carry, x):
        chol_i, rot_chol_i = x
        gl_i = jnp.einsum("pj,ji->pi", green, chol_i, optimize="optimal")
        lci2_green_i = jnp.einsum(
            "pi,ji->pj", rot_chol_i, ci2_green, optimize="optimal"
        )
        carry[0] += 0.5 * jnp.einsum(
            "pi,pi->", gl_i, lci2_green_i, optimize="optimal"
        )
        glgp_i = jnp.einsum("pi,it->pt", gl_i, greenp, optimize="optimal").astype(
            jnp.complex64
        )
        l2ci2_1 = jnp.einsum(
            "pt,qu,ptqu->",
            glgp_i,
            glgp_i,
            ci2.astype(jnp.float32),
            optimize="optimal",
        )
        l2ci2_2 = jnp.einsum(
            "pu,qt,ptqu->",
            glgp_i,
            glgp_i,
            ci2.astype(jnp.float32),
            optimize="optimal",
        )
        carry[1] += 2 * l2ci2_1 - l2ci2_2
        return carry, 0.0

    [e2_2_2_2, e2_2_3], _ = lax.scan(scanned_fun, [0.0, 0.0], (chol, rot_chol))
    e2_2_2 = 4 * (e2_2_2_1 + e2_2_2_2)

    e2_2 = e2_2_1 + e2_2_2 + e2_2_3

    e2 = e2_0 + e2_1 + e2_2

    # overlap
    overlap_1 = 2 * ci1g  # jnp.einsum("ia,ia", ci1, green_occ)
    overlap_2 = gci2g
    overlap = 1.0 + overlap_1 + overlap_2
    olp_inv = 1 - 2*ci1g - gci2g + (2*ci1g)**2
    
    e_pert = (e1 + e2) * olp_inv + e0
    e_orig = (e1 + e2) / overlap + e0

    # return (e1 + e2) / overlap + e0
    return e_pert, e_orig

In [26]:
def thouless_trans(t1):
    q, r = np.linalg.qr(t1)
    u_ai = r.T
    u_ji = q
    u_occ = np.vstack((u_ji,u_ai))
    u, _, _ = np.linalg.svd(u_occ)
    return u

In [34]:
t1 = wave_data["ci1"]
nocc = trial.nelec[0]
u_t1 = thouless_trans(t1)
u_occ = u_t1[:,:nocc]
wave_data['u_t1'] = u_occ

In [ ]:
# from ad_afqmc import wavefunctions
# calc_e_rhf = wavefunctions.rhf(trial.norb, trial.nelec,n_batch=trial.n_batch
#                                 )._calc_energy_restricted
# calc_e_rhf()

In [35]:
prop_data['walkers'][0]

Array([[-0.00627359-0.0258082j , -0.00540077-0.03404597j,
        -0.99654127+0.03533954j],
       [ 0.01037218+0.05235042j, -0.99000166+0.06742266j,
        -0.00555546-0.03405047j],
       [ 0.98774895-0.08190872j, -0.010567  -0.05228849j,
         0.0048656 +0.02586054j],
       [-0.01329229-0.09167786j, -0.01146547-0.05796769j,
         0.00551622+0.02270191j],
       [ 0.01057563+0.06127817j,  0.01096494+0.07863907j,
        -0.00631271-0.03615039j],
       [ 0.00680521+0.03993385j,  0.00572811+0.03857449j,
        -0.00366554-0.04321949j]], dtype=complex128)

In [38]:
print(u_occ.shape)

(6, 3)


In [69]:
walker = prop_data['walkers'][0]
green = (walker.dot(jnp.linalg.inv(walker[:nocc, :]))).T
# green = (walker.dot(jnp.linalg.inv(u_occ.T @ walker))) @ u_occ.T
print(green.shape)
print(green[:nocc,nocc:])

(3, 6)
[[ 1.39059730e-17+0.j  1.43713517e-16+0.j -3.71730138e-16+0.j]
 [-9.02056208e-17+0.j -1.66533454e-16+0.j  5.31171182e-33+0.j]
 [-4.74822782e-17+0.j -1.59134048e-16+0.j  4.91004945e-17+0.j]]


In [70]:
ci1 = wave_data["ci1"]
g_ia = green[:nocc,nocc:]
ci1g = jnp.einsum("ia,ia->", ci1, g_ia, optimize="optimal")
print(ci1g)

(6.539875472771294e-19+0j)


In [71]:
ci2g_c = jnp.einsum("iajb,ia->jb", ci2, g_ia)
ci2g_e = jnp.einsum("iajb,ib->ja", ci2, g_ia)
ci2g = 2 * ci2g_c - ci2g_e
gci2g = jnp.einsum("ia,ia->", ci2g, g_ia,optimize="optimal")
print(gci2g)

(-2.846813597073128e-32+0j)


In [68]:
ci1g*2+gci2g

Array(3.15812164e-05+0.j, dtype=complex128)

In [ ]:
from jax import lax
def _calc_ccsd_walker_pert_energy(trial, walker, ham_data, wave_data):
    ci1, ci2, u_t1 = wave_data["ci1"], wave_data["ci2"], wave_data['u_t1']
    ci2 = ci2 - jnp.einsum('ia,jb->iajb',ci1,ci1)
    ci1 = np.zeros_like(ci1)
    # ci2 = np.zeros_like(ci2)
    nocc = trial.nelec[0]
    green = (walker.dot(jnp.linalg.inv(u_t1.T @ walker))) @ u_t1.T
    green = green[:nocc,:]
    green_occ = green[:, nocc:].copy()
    greenp = jnp.vstack((green_occ, -jnp.eye(trial.norb - nocc)))

    chol = ham_data["chol"].reshape(-1, trial.norb, trial.norb)
    rot_chol = chol[:, : trial.nelec[0], :]
    h1 = (ham_data["h1"][0] + ham_data["h1"][1]) / 2.0
    hg = jnp.einsum("pj,pj->", h1[:nocc, :], green)

    # 0 body energy
    e0 = ham_data["h0"]

    # 1 body energy
    # ref
    e1_0 = 2 * hg

    # single excitations
    # ci1g = jnp.einsum("ia,ia->", ci1, green_occ, optimize="optimal")
    # e1_1_1 = 4 * ci1g * hg
    # gpci1 = greenp @ ci1.T
    # ci1_green = gpci1 @ green
    # e1_1_2 = -2 * jnp.einsum("ij,ij->", h1, ci1_green, optimize="optimal")
    # e1_1 = e1_1_1 + e1_1_2

    # double excitations
    # ci2g_c = jnp.einsum("ptqu,pt->qu", ci2, green_occ)
    ci2g_c = jnp.einsum("iajb,ia->jb", ci2, green_occ)
    ci2g_e = jnp.einsum("iajb,ib->ja", ci2, green_occ)
    ci2_green_c = (greenp @ ci2g_c.T) @ green
    ci2_green_e = (greenp @ ci2g_e.T) @ green
    ci2_green = 2 * ci2_green_c - ci2_green_e
    ci2g = 2 * ci2g_c - ci2g_e
    gci2g = jnp.einsum("ia,ia->", ci2g, green_occ, optimize="optimal")
    e1_2_1 = 2 * hg * gci2g
    e1_2_2 = -2 * jnp.einsum("ij,ij->", h1, ci2_green, optimize="optimal")
    e1_2 = e1_2_1 + e1_2_2
    # e1 = e1_0 + e1_1 + e1_2
    e1 = e1_0 + e1_2

    # two body energy
    # ref
    lg = jnp.einsum("gpj,pj->g", rot_chol, green, optimize="optimal")
    # lg1 = jnp.einsum("gpj,pk->gjk", rot_chol, green, optimize="optimal")
    lg1 = jnp.einsum("gpj,qj->gpq", rot_chol, green, optimize="optimal")
    e2_0_1 = 2 * lg @ lg
    e2_0_2 = -jnp.sum(vmap(lambda x: x * x.T)(lg1))
    e2_0 = e2_0_1 + e2_0_2

    # single excitations
    # e2_1_1 = 2 * e2_0 * ci1g
    # lci1g = jnp.einsum("gij,ij->g", chol, ci1_green, optimize="optimal")
    # e2_1_2 = -2 * (lci1g @ lg)
    # lci1g1 = jnp.einsum("gij,jk->gik", chol, ci1_green, optimize="optimal")
    # glgpci1 = jnp.einsum(("gpi,iq->gpq"), gl, gpci1, optimize="optimal")
    # ci1g1 = ci1 @ green_occ.T
    # e2_1_3 = jnp.einsum("gpq,gpq->", glgpci1, lg1, optimize="optimal")
    # e2_1_3_1 = jnp.einsum("gpq,gqr,rp->", lg1, lg1, ci1g1, optimize="optimal")
    # lci1g = jnp.einsum("gip,qi->gpq", ham_data["lci1"], green, optimize="optimal")
    # e2_1_3_2 = -jnp.einsum("gpq,gqp->", lci1g, lg1, optimize="optimal")
    # e2_1_3 = e2_1_3_1 + e2_1_3_2
    # e2_1 = e2_1_1 + 2 * (e2_1_2 + e2_1_3)

    # double excitations
    e2_2_1 = e2_0 * gci2g
    lci2g = jnp.einsum("gij,ij->g", chol, ci2_green, optimize="optimal")
    e2_2_2_1 = -lci2g @ lg

    # lci2g1 = jnp.einsum("gij,jk->gik", chol, ci2_green, optimize="optimal")
    # lci2_green = jnp.einsum("gpi,ji->gpj", rot_chol, ci2_green, optimize="optimal")
    # e2_2_2_2 = 0.5 * jnp.einsum("gpi,gpi->", gl, lci2_green, optimize="optimal")
    def scanned_fun(carry, x):
        chol_i, rot_chol_i = x
        gl_i = jnp.einsum("pj,ji->pi", green, chol_i, optimize="optimal")
        lci2_green_i = jnp.einsum(
            "pi,ji->pj", rot_chol_i, ci2_green, optimize="optimal"
        )
        carry[0] += 0.5 * jnp.einsum(
            "pi,pi->", gl_i, lci2_green_i, optimize="optimal"
        )
        glgp_i = jnp.einsum("pi,it->pt", gl_i, greenp, optimize="optimal").astype(
            jnp.complex64
        )
        l2ci2_1 = jnp.einsum(
            "pt,qu,ptqu->",
            glgp_i,
            glgp_i,
            ci2.astype(jnp.float32),
            optimize="optimal",
        )
        l2ci2_2 = jnp.einsum(
            "pu,qt,ptqu->",
            glgp_i,
            glgp_i,
            ci2.astype(jnp.float32),
            optimize="optimal",
        )
        carry[1] += 2 * l2ci2_1 - l2ci2_2
        return carry, 0.0

    [e2_2_2_2, e2_2_3], _ = lax.scan(scanned_fun, [0.0, 0.0], (chol, rot_chol))
    e2_2_2 = 4 * (e2_2_2_1 + e2_2_2_2)

    e2_2 = e2_2_1 + e2_2_2 + e2_2_3

    # e2 = e2_0 + e2_1 + e2_2
    e2 = e2_0 + e2_2

    # overlap
    # overlap_1 = 2 * ci1g  # jnp.einsum("ia,ia", ci1, green_occ)
    overlap_2 = gci2g
    # overlap = 1.0 + overlap_1 + overlap_2
    overlap = 1.0 + overlap_2
    # olp_inv = 1 - 2*ci1g - gci2g + (2*ci1g)**2
    olp_inv = 1 - gci2g
    
    e_pert = (e1 + e2) * olp_inv + e0
    e_orig = (e1 + e2) / overlap + e0

    # return (e1 + e2) / overlap + e0
    return e_pert, e_orig, gci2g

In [18]:
print(mycc.e_tot)

-2.9998514912772754


In [73]:
e1,e2 = _calc_cisd_walker_pert_energy(trial, prop_data['walkers'][0], ham_data, wave_data)
print(e1)
print(e2)
print(e1-e2)

(-2.999851469512132+0j)
(-2.999851469512132+0j)
0j


In [76]:
prop_data, (block_energy, block_weight) \
    = sampler._block_scan(prop_data,None,ham_data,prop,trial,wave_data)

In [55]:
trial._calc_energy_restricted(prop_data['walkers'][0], ham_data, wave_data)

Array(-2.99985147+0.j, dtype=complex128)

In [91]:
t2_0 = np.zeros(mycc.t2.shape)
eris = mycc.ao2mo(mycc.mo_coeff)
eccsd = mycc.energy(mycc.t1, t2_0, eris)
ecc_t2_0 = eccsd+mf.e_tot
print(ecc_t2_0,eccsd)

-2.7501475540010003 2.4901828913079822e-06


In [89]:
e1,e2,c2gg = _calc_ccsd_walker_pert_energy(trial, prop_data['walkers'][0], ham_data, wave_data)
print(e1,e2)
print(1-c2gg,1/(1+c2gg))

(-2.750147553891801+0j) (-2.750147553891801+0j)
(1+0j) (1+0j)


In [84]:
1/(1+c2gg)

Array(1.00000388+1.6543086e-09j, dtype=complex128)

In [83]:
1-c2gg

Array(1.00000388+1.65429576e-09j, dtype=complex128)

In [78]:
_calc_cisd_walker_pert_energy(trial, prop_data['walkers'][0], ham_data, wave_data)

(Array(-2.99733417-0.00029331j, dtype=complex128),
 Array(-2.9981479-0.00015193j, dtype=complex128))